In [2]:
import time

import torch
from transformers import AutoTokenizer
from layerskip_modeling.modeling_layerskip_gemma2 import LayerSkipGemma2ForCausalLM

pretrained_model_name_or_path = "../models/google--gemma-2-2b-it"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path)
# model = LayerSkipGemma2ForCausalLM.from_pretrained(pretrained_model_name_or_path, torch_dtype=torch.bfloat16).to(device)

In [4]:
model.config.num_hidden_layers

26

In [8]:
skip_layer_ids = {
    "attn": [
        2,
        5,
        10,
        18,
    ],
    "mlp": [
        2,
        5,
        10,
        18,
    ]
}

model.set_skip_layer_ids(skip_layer_ids=skip_layer_ids)

skip_layer_ids: {'attn': [2, 5, 10, 18], 'mlp': [2, 5, 10, 18]}


In [9]:
messages = [
    [
        {
            "role": "user",
            "content": "What is the capital of Taiwan. And why?",
        },
    ],
]


# Tokenize
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding=True,
).to(device)

In [10]:
prompt_token_num = inputs["input_ids"].shape[1]
prompt_token_num

17

In [11]:
model.set_draft_mode(False)

start_time = time.time()

outputs = model.generate(**inputs, max_new_tokens=512)
total_token_num = outputs[0].shape[0]
completion_token_num = total_token_num - prompt_token_num
cost_time = time.time() - start_time

token_per_second = completion_token_num / cost_time
response = tokenizer.batch_decode(outputs)[0]

print(response)
print()
print(f"Completion Token Number: {completion_token_num}")
print(f"Cost Time: {cost_time}, Speed: {token_per_second} token/sec")

<bos><bos><start_of_turn>user
What is the capital of Taiwan. And why?<end_of_turn>
 
The capital of Taiwan is **Taipei**. 

Here's why:

* **Official Designation:** Taipei is the official capital of the Republic of China (ROC), which governs Taiwan. 
* **Historical Significance:**  Taipei has been the capital of Taiwan since the ROC government moved there in 1949 after the Chinese Civil War.
* **Economic and Cultural Hub:** Taipei is a major economic and cultural center in Taiwan, with a thriving business district, vibrant nightlife, and numerous museums, art galleries, and historical sites. 
* **International Presence:**  Taipei is a major international hub for trade and tourism, hosting numerous embassies and consulates.


Let me know if you have any other questions about Taiwan! 
<end_of_turn>

Completion Token Number: 157
Cost Time: 4.492177486419678, Speed: 34.94964312399219 token/sec


In [12]:
model.set_draft_mode(True)

start_time = time.time()

outputs = model.generate(**inputs, max_new_tokens=512)
total_token_num = outputs.shape[-1]
completion_token_num = total_token_num - prompt_token_num
cost_time = time.time() - start_time

token_per_second = completion_token_num / cost_time
response = tokenizer.batch_decode(outputs)[0]

print(response)
print()
print(f"Completion Token Number: {completion_token_num}")
print(f"Cost Time: {cost_time}, Speed: {token_per_second} token/sec")

<bos><bos><start_of_turn>user
What is the capital of Taiwan. And why?<end_of_turn>
* **Capital of Taiwan** is the capital of Taiwan. 

**Answer:**

**Taipei**

**Explanation:**

Taipei is the capital of Taiwan. 

**Explanation:**

Taipei is the capital of Taiwan. 

**Answer:**

**Taipei** is the capital of Taiwan. 

**Explanation:**

Taipei is the capital of Taiwan. 

**Answer:**

**Taipei** is the capital of Taiwan. 

**Explanation:**

Taipei is the capital of Taiwan. 

**Answer:**

**Taipei** is the capital of Taiwan. 

**Explanation:**

Taipei is the capital of Taiwan. 

**Answer:**

**Taipei** is the capital of Taiwan. 

**Explanation:**

Taipei is the capital of Taiwan. 

**Answer:**

**Taipei** is the capital of Taiwan. 

**Explanation:**

Taipei is the capital of Taiwan. 

**Answer:**

**Taipei** is the capital of Taiwan. 

**Explanation:**

Taipei is the capital of Taiwan. 

**Answer:**

**Taipei** is the capital of Taiwan. 

**Explanation:**

Taipei is the capital of Taiwan. 


In [1]:
from datasets import load_dataset


dataset = load_dataset("shibing624/sharegpt_gpt4")

Generating train split: 100%|██████████| 103415/103415 [00:00<00:00, 144630.59 examples/s]


In [9]:
from pprint import pprint

pprint(dataset["train"]["conversations"][0])

[{'from': 'human',
  'value': "Summarize the main ideas of Jeff Walker's Product Launch Formula "
           'into bullet points as it pertains to a growth marketing agency '
           'implementing these strategies and tactics for their clients...'},
 {'from': 'gpt',
  'value': "Here are the main ideas of Jeff Walker's Product Launch Formula "
           'that can be applied by a growth marketing agency for their '
           'clients:\n'
           '\n'
           '1. Identify the target audience and their needs: Understand the '
           'ideal customer for the product or service, and create a messaging '
           'that resonates with them.\n'
           '2. Pre-launch: Build anticipation and excitement for the launch by '
           'creating buzz, gathering testimonials and case studies, and using '
           'social media to create awareness.\n'
           '3. Launch: Use a well-crafted launch sequence to maximize sales '
           'and conversions. This can include offeri

In [21]:
samples = dataset["train"]["conversations"][:100]
samples[0]

[{'from': 'human',
  'value': "Summarize the main ideas of Jeff Walker's Product Launch Formula into bullet points as it pertains to a growth marketing agency implementing these strategies and tactics for their clients..."},
 {'from': 'gpt',
  'value': "Here are the main ideas of Jeff Walker's Product Launch Formula that can be applied by a growth marketing agency for their clients:\n\n1. Identify the target audience and their needs: Understand the ideal customer for the product or service, and create a messaging that resonates with them.\n2. Pre-launch: Build anticipation and excitement for the launch by creating buzz, gathering testimonials and case studies, and using social media to create awareness.\n3. Launch: Use a well-crafted launch sequence to maximize sales and conversions. This can include offering bonuses, creating scarcity, and using a deadline to create urgency.\n4. Post-launch: Follow up with customers, gather feedback, and continue to provide value to keep them engaged 

In [22]:
samples[0][-2]

{'from': 'human',
 'value': 'Summarize the 6 human needs as Tony Robbins explains...'}

In [ ]:
samples = dataset["train"]["conversations"][:100]
samples = [[{"role": conversation["from"].replace("human", "user").replace("gpt", "assistant"), "content": conversation["value"]} for conversation in sample] for sample in samples]

In [24]:
samples[0]

[{'role': 'user',
  'content': "Summarize the main ideas of Jeff Walker's Product Launch Formula into bullet points as it pertains to a growth marketing agency implementing these strategies and tactics for their clients..."},
 {'role': 'assistant',
  'content': "Here are the main ideas of Jeff Walker's Product Launch Formula that can be applied by a growth marketing agency for their clients:\n\n1. Identify the target audience and their needs: Understand the ideal customer for the product or service, and create a messaging that resonates with them.\n2. Pre-launch: Build anticipation and excitement for the launch by creating buzz, gathering testimonials and case studies, and using social media to create awareness.\n3. Launch: Use a well-crafted launch sequence to maximize sales and conversions. This can include offering bonuses, creating scarcity, and using a deadline to create urgency.\n4. Post-launch: Follow up with customers, gather feedback, and continue to provide value to keep them

In [25]:
processed_samples = [
    tokenizer(
        tokenizer.apply_chat_template(sample, tokenize=False),
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding=True,
    )
    for sample in samples
]

In [31]:
lengths = []
for sample in processed_samples:
    lengths.append(sample["input_ids"].shape[-1])

In [1]:
import json

skip_layer_ids = {'attn': [1, 2, 8, 12, 16, 18, 19, 21, 22, 23, 27, 30, 31, 32, 35, 36, 37, 40], 'mlp': [4, 8, 9, 11, 12, 14, 15, 17, 18, 19, 25, 27, 28, 29, 31, 33, 36, 37, 38, 40, 41]}

In [2]:
with open("skip_layer_ids.json", "w") as f:
    json.dump(skip_layer_ids, f)